In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()

import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

In [3]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/readerpri.c", 
    lang="", 
    number=True,
    title="<b>readerpri.c - Full source code for readers/writers solution with semaphores</b>",
    h="100%", 
    w="100%"
) + '</font>'))


<font size="1.2rem"><b>readerpri.c - Full source code for readers/writers solution with semaphores</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <pthread.h>
1: #include <stdlib.h>
2: #include <sys/types.h>
3: #include <unistd.h>
4: #include <string.h>
5: #include <stdio.h>
6: #include <time.h>
7: #include <bsd/sys/time.h>
8: #include <stdbool.h>
9: #include <stdatomic.h>
10: #include <semaphore.h>
11: 
12: atomic_bool started = false;
13: 
14: /* Begin Courtois et al. reader/writer semaphore solution with reader priority here */
15: static void read_db();
16: static void write_db();
17: 
18: sem_t mutex;         /* Initially 1 */
19: sem_t counter_mutex; /* Initially 1 */
20: int reader_count = 0;
21: 
22: static void writer()
23: {
24:     sem_wait(&mutex);
25:     write_db();
26:     sem_post(&mutex);
27: }
28: 
29: static void reader()
30: {
31:     
32:     sem_wait(&counter_mutex);
33:     reader_count++;
34:     if (reader_count == 1) {
35:         /* first reader synchronizes with writers to acquire the resource */
36:         sem_wait(&mutex);
37:     }
38:     sem_post(&counter_mutex);
39:     
40:     read_db();
41:     
42:     sem_wait(&counter_mutex);
43:     reader_count--;
44:     if (reader_count == 0) {
45:         /* last reader releases the resource */
46:         sem_post(&mutex);
47:     }
48:     sem_post(&counter_mutex);
49: }
50: /* End Courtois et al. reader/writer semaphore solution with reader priority. */
51: 
52: static void *writer_thread(void *arg)
53: {
54:     long niters = (long)arg;
55:     long i;
56:     
57:     while(!started) ; /* spin until all threads are ready */
58:         
59:     for (i = 0; i < niters; i++) {
60:         writer();
61:     }
62:     return (void *)0;
63: }
64: 
65: static void *reader_thread(void *arg)
66: {
67:     long niters = (long)arg;
68:     long i;
69:     
70:     while(!started) ; /* spin until all threads are ready */
71: 
72:     for (i = 0; i < niters; i++) {
73:         reader();
74:     }
75:     return (void *)0;
76: }
77: 
78: int main(int argc, char **argv)
79: {
80:     pthread_t *tids;
81:     long niters;
82:     int nreaders, nwriters, nthreads, i;
83:     
84:     if (argc != 4) {
85:         fprintf(stderr,"Usage: %s <num_readers> <num_writers> <num_iters>\n",argv[0]);
86:         return 1;
87:     }
88: 
89:     nreaders = atoi(argv[1]);
90:     nwriters = atoi(argv[2]);
91:     niters = atol(argv[3]);
92:     
93:     printf("Main thread: Beginning test with %d readers and %d writers, each doing %ld iterations\n", 
94:            nreaders, nwriters, niters);
95:     sem_init(&counter_mutex, 0, 1);
96:     sem_init(&mutex, 0, 1);
97:     
98:     nthreads = nreaders + nwriters;
99:     tids = (pthread_t *)malloc(nthreads * sizeof(pthread_t));
100:     
101:     /* We create the requested number of reader and writer threads, each doing the same number of iterations. 
102:      * When all threads have completed, we report the throughput. 
103:      */
104:     for (i = 0; i < nreaders; i++) {
105:         (void)pthread_create(&tids[i], NULL, reader_thread, (void *)niters );
106:     }
107:     
108:     for(i = 0; i < nwriters; i++) {
109:         (void)pthread_create(&tids[i+nreaders], NULL, writer_thread, (void *)niters );
110:     }
111:     
112:     struct timespec strt;
113:     struct timespec end;
114:     clock_gettime(CLOCK_MONOTONIC_RAW, &strt);
115:     started = true;
116:     /* Wait for child threads to finish */
117:     for (i = 0; i < nthreads; i++) {
118:         pthread_join(tids[i], NULL);
119:     }
120:     clock_gettime(CLOCK_MONOTONIC_RAW, &end);
121:     
122:     /* report performance statistics */    
123:     struct timespec diff;
124:     timespecsub(&end, &strt, &diff);
125:     float elapsed_sec = diff.tv_sec + diff.tv_nsec / 1000000000.0;
126:     printf("Main thread: Completed %ld iterations in %f seconds (%f iters per second)\n",
127:           niters*nthreads, elapsed_sec, niters*nthreads*1.0/elapsed_sec);
128: 
129:     return 0;
130: }
131: 
132: static void read_db() 
133: {
134:     return;
135: }
136: 
137: static void write_db()
138: {
139:     return;
140: }

```


</div>
</font>

In [4]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/rwlock.h", 
    lang="", 
    number=True,
    title="<b>rwlock.h - Source code for reader/writer locks using semaphores</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>rwlock.h - Source code for reader/writer locks using semaphores</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: #include <semaphore.h>
1: 
2: typedef struct rwlock_s {
3:     sem_t mutex;
4:     sem_t counter_mutex;
5:     int reader_count;
6: } rwlock_t; 
7: 
8: static inline void rwlock_init(rwlock_t *l) {
9:     l->reader_count = 0;
10:     sem_init(&l->counter_mutex, 0, 1);
11:     sem_init(&l->mutex, 0, 1);
12: }
13: 
14: static inline void rwlock_acquire_write(rwlock_t *l)
15: {
16:     sem_wait(&l->mutex);
17: }
18: 
19: static inline void rwlock_release_write(rwlock_t *l)
20: {
21:     sem_post(&l->mutex);
22: }
23: 
24: static inline void rwlock_acquire_read(rwlock_t *l)
25: {
26:     sem_wait(&l->counter_mutex);
27:     l->reader_count++;
28:     if (l->reader_count == 1) {
29:         /* first reader synchronizes with writers to acquire the resource */
30:         sem_wait(&l->mutex);
31:     }
32:     sem_post(&l->counter_mutex);   
33: }
34: 
35: static inline void rwlock_release_read(rwlock_t *l)
36: {
37:     sem_wait(&l->counter_mutex);
38:     l->reader_count--;
39:     if (l->reader_count == 0) {
40:         /* last reader releases the resource */
41:         sem_post(&l->mutex);
42:     }
43:     sem_post(&l->counter_mutex);
44: }

```


</div>
</font>